# Pymaceuticals Inc.
---

### Analysis

1) There was a fairly equivalent number of male and female mice used in this study.
2) The most frequent, as well as the most successful, drug regimen used was Capomulin, while the least frequent drug regimen was Propriva.
3) There is a strong correlation between mouse weight and the average tumor volume since the correlation coefficient is 0.84.

In [ ]:
# Dependencies and Setup
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st
import numpy as np

# Study data files
mouse_metadata_path = "../Resources/Mouse_metadata.csv"
study_results_path = "../Resources/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
combined_data = pd.merge(study_results, mouse_metadata, how='left', on="Mouse ID")

# Display the data table for preview
combined_data.head()

In [ ]:
# Checking the number of mice.
number_mice = len(combined_data['Mouse ID'].unique())
number_mice

In [ ]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 
dup_mouse_ID = combined_data.loc[combined_data.duplicated(subset=["Mouse ID", "Timepoint",]), "Mouse ID"].unique()
dup_mouse_ID

In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
dup_mice_rows = combined_data[combined_data["Mouse ID"] == "g989"]
dup_mice_rows

In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
clean_df = combined_data.loc[(combined_data["Mouse ID"] != "g989"), :]
clean_df

In [ ]:
# Checking the number of mice in the clean DataFrame.
number_clean_mice = len(clean_df['Mouse ID'].unique())
number_clean_mice

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
tumor_volume_mean = clean_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].mean()
tumor_volume_median = clean_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].median()
tumor_volume_var = clean_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].var()
tumor_volume_std = clean_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].std()
tumor_volume_sem = clean_df.groupby(["Drug Regimen"])["Tumor Volume (mm3)"].sem()
# Assemble the resulting series into a single summary DataFrame.
summary = pd.DataFrame({'Mean Tumor Volume': tumor_volume_mean,
                        'Median Tumor Volume': tumor_volume_median,
                        'Tumor Volume Variance': tumor_volume_var,
                        'Tumor Volume Std. Dev.': tumor_volume_std,
                        'Tumor Volume Std. Err.': tumor_volume_sem
                       })
summary

In [ ]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line
one_line_summary = clean_df.groupby(['Drug Regimen']).agg({"Tumor Volume (mm3)": ["mean", "median", "var", "std", "sem"]})
one_line_summary

## Bar and Pie Charts

In [ ]:
drug_count = clean_df['Drug Regimen'].value_counts()

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.
drug_count.plot(kind = "bar")
plt.xlabel("Drug Regimen")
plt.ylabel("# of Observed Mouse Timepoints")
plt.show()

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.
plt.bar(drug_count.index, drug_count.values)
plt.xticks(rotation=90)
plt.xlabel("Drug Regimen")
plt.ylabel("# of Observed Mouse Timepoints")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
mouse_gen = clean_df["Sex"].value_counts()
mouse_gen.plot(kind="pie", autopct="%1.1f%%")
plt.ylabel("Sex")
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot
plt.pie(mouse_gen, labels=mouse_gen.index, autopct="%1.1f%%")
plt.ylabel("Sex")
plt.show()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin
final_tumor_vol = clean_df.groupby(["Mouse ID"])["Timepoint"].max()
final_tumor_vol = final_tumor_vol.reset_index()
# Start by getting the last (greatest) timepoint for each mouse
last_tp = clean_df.groupby(["Mouse ID"])["Timepoint"].max()
last_tp = last_tp.reset_index()

# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint
merge_data = last_tp.merge(clean_df, on=["Mouse ID", "Timepoint"], how="left")

In [ ]:
# Put treatments into a list for for loop (and later for plot labels)
treatments = ["Capomulin", "Ramicane", "Infubinol", "Ceftamin"]

# Create empty list to fill with tumor vol data (for plotting)
tumor_vol_data = []

# Calculate the IQR and quantitatively determine if there are any potential outliers. 
for drug in treatments:
    
    # Locate the rows which contain mice on each drug and get the tumor volumes
    tumor_volume = merge_data.loc[merge_data["Drug Regimen"]==drug, "Tumor Volume (mm3)"]
    
    # add subset 
    tumor_vol_data.append(tumor_volume)
    
    # Determine outliers using upper and lower bounds
    quartiles = tumor_volume.quantile([.25,.5,.75])
    lowerq = quartiles[0.25]
    upperq = quartiles[0.75]
    iqr = upperq-lowerq
    
    lower_bound = lowerq - (1.5*iqr)
    upper_bound = upperq + (1.5*iqr)
    
    outliers = tumor_volume.loc[(tumor_volume<lower_bound) | (tumor_volume>upper_bound)]
    print(f"{drug}'s potential outliers: {outliers}")

In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.
fig1, ax1 = plt.subplots()
ax1.set_ylabel('Final Tumor Volume (mm3)')
ax1.boxplot(tumor_vol_data, labels=treatments, flierprops = dict(markerfacecolor = "red", markersize="12"))
plt.show()

## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin
capomulin_treatment_df = clean_df.loc[clean_df["Drug Regimen"]=="Capomulin",:]
capomulin_mouse = capomulin_treatment_df.loc[capomulin_treatment_df["Mouse ID"]=="l509",:]
timepoint_days = capomulin_mouse["Timepoint"]
tumor_volume_mm3 = capomulin_mouse["Tumor Volume (mm3)"]
fig2, ax2 = plt.subplots()
plt.plot(timepoint_days, tumor_volume_mm3, linewidth=2, color="blue")
plt.title("Capomulin treatment of mouse l509")
plt.xlabel('Timepoint (days)')
plt.ylabel('Tumor Volume (mm3)')
plt.show()

In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen
fig1, ax1 = plt.subplots()
av_tumor_vol = capomulin_treatment_df.groupby(['Mouse ID'])[['Weight (g)', 'Tumor Volume (mm3)']].mean()
av_tumor_vol
plt.scatter(av_tumor_vol['Weight (g)'], av_tumor_vol['Tumor Volume (mm3)'])
plt.xlabel('Weight (g)')
plt.ylabel('Average Tumor Volume (mm3)')
plt.show()

## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
correlation = round(st.pearsonr(av_tumor_vol['Weight (g)'], av_tumor_vol['Tumor Volume (mm3)'])[0],2)
print(f'The correlation between mouse weight and the average tumor volume is {correlation}')

slope, int, r, p, std_err = st.linregress(av_tumor_vol['Weight (g)'], av_tumor_vol['Tumor Volume (mm3)'])
lin_reg = av_tumor_vol['Weight (g)'] * slope + int

plt.scatter(av_tumor_vol['Weight (g)'], av_tumor_vol['Tumor Volume (mm3)'])
plt.plot(av_tumor_vol['Weight (g)'], lin_reg, color='red')
plt.xlabel('Weight (g)')
plt.ylabel('Average Tumor Volume (mm3)')
plt.show()